In [1]:
import psycopg2
from psycopg2 import sql
import re 
import json
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
from tkinter import Tk
from tkinter import filedialog
import csv
import io
hostname = 'localhost'
database = 'First_test'
username = 'postgres'
pwd = 'Yann'
port_id = 5432

In [2]:
import pandas as pd
import json
import csv
import openpyxl

import xml.etree.ElementTree as ET
from tkinter import Tk, filedialog
import psycopg2

hostname = 'localhost'
database = 'First_test'
username = 'postgres'
pwd = 'Yann'
port_id = 5432

def connect_to_database():
    try:
        conn = psycopg2.connect(
            host=hostname,
            dbname=database,
            user=username,
            password=pwd,
            port=port_id
        )
        return conn
    except Exception as error:
        print(error)
        return None


import re






def sanitize_column_name(name):
    # Replace non-alphanumeric characters with underscores
    sanitized_name = re.sub(r'\W+', '_', name)
    # Remove leading or trailing whitespace characters
    sanitized_name = sanitized_name.strip()
    # Remove underscores from date-like strings
   
    return sanitized_name


def df_to_sql():
    conn = connect_to_database()
    root = Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(initialdir="C:/Users/YourUsername/Desktop", title="Select file",
                                           filetypes=(("CSV files", "*.csv"), ("XML files", "*.xml"), ("Excel files", "*.xlsx")))
    table_name = input("Enter the table name: ")
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.xml'):
            tree = ET.parse(file_path)
            root = tree.getroot()
            data = []
            for elem in root:
                data.append(elem.attrib)
            df = pd.DataFrame(data)
        elif file_path.endswith('.xlsx'):  # Add handling for Excel files
            df = pd.read_excel(file_path)
        else:
            print("Unsupported file format.")
            return

        # Add 'id' column with sequential values
        df.insert(0, 'id', range(1, 1 + len(df)))

        # Sanitize column names to comply with SQL naming conventions
        df.columns = [sanitize_column_name(col) for col in df.columns]

        # Convert DataFrame to a list of tuples (each tuple represents a row)
        data_list = [tuple(row) for row in df.to_numpy()]

        # Create table if it does not exist
        with conn.cursor() as cur:
            # Handle column names with special characters or spaces
            create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join([f'{col} VARCHAR' for col in df.columns])})"
            cur.execute(create_table_query)

            # Insert data into the table
            for row in data_list:
                row = list(row)
                row[-1] = str(row[-1])  # Convert last element (nested dict) to string
                # Handle column names with special characters or spaces
                insert_query = f"INSERT INTO {table_name} ({', '.join([f'{col}' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(df.columns))})"
                cur.execute(insert_query, row)

        conn.commit()
        print(f"DataFrame from {file_path} successfully saved to table {table_name} in the database.")
    except Exception as e:
        print("Error:", e)
    finally:
        if conn:
            conn.close()








# Example usage:
if __name__ == "__main__":
    df_to_sql()


Error: ERREUR:  élément indésirable après la valeur numérique sur ou près de « 2018_ »
LINE 1: ...e VARCHAR, RegionType VARCHAR, StateName VARCHAR, 2018_03_31...
                                                             ^



In [3]:
df = pd.read_csv(r"C:\Users\yskon\Desktop\Desk_App\Data\Zillow_CSV\County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,202439.173324,...,776848.290023,773099.225984,775250.442573,781373.168323,791486.555040,804039.376556,816650.613722,826916.517556,834627.929107,839408.798822
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,142230.761086,...,271789.156474,273195.319186,274679.762036,276346.208345,278195.985808,280427.994007,282293.720706,283691.589240,284563.214653,285001.791708
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,107763.281727,...,274539.541373,273977.274584,273999.684086,274650.233887,275424.434724,276226.228326,276558.372120,276601.979839,276354.918892,276068.242700
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,141146.256502,...,443439.620104,442430.540828,443183.037678,445096.958351,447789.735064,450706.610659,453168.669276,455321.626806,457080.234374,458025.226499
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,213074.494470,...,824423.976916,827521.754957,834158.592031,843452.426540,854618.092740,866300.086332,877589.314100,886848.996547,893220.009056,896579.371655


In [4]:
df.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS', '2000-01-31',
       ...
       '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31',
       '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31'],
      dtype='object', length=297)

In [5]:
def connect_to_database():
    try:
        conn = psycopg2.connect(
            host=hostname,
            dbname=database,
            user=username,
            password=pwd,
            port=port_id
        )
        return conn
    except Exception as error:
        print(error)
        return None
    


In [6]:
def get_data(limit=None):
    table_name = input("Enter the table name: ")
    while not isinstance(table_name, str):
        print("Table name must be a string.")
        table_name = input("Enter the table name: ")
    conn = connect_to_database()
    if conn:
        try:
            with conn.cursor() as cur:
                if limit:
                    cur.execute(sql.SQL("SELECT * FROM public.{} LIMIT %s;").format(sql.Identifier(table_name)), (limit,))
                else:
                    cur.execute(sql.SQL("SELECT * FROM public.{};").format(sql.Identifier(table_name)))
                rows = cur.fetchall()
                column_names = [desc[0] for desc in cur.description]
                
                df = pd.DataFrame(rows, columns=column_names)
                return df
        except Exception as error:
            print(error)
        finally:
            conn.close()



In [7]:
df = get_data()
    
    # Créer le DataFrame

    
if df is not None:
    print("DataFrame created successfully:")
    print(df.head())
else:
    print("No data available.")

DataFrame created successfully:
   id        date 2b__high__USD_ Digital_Currency_Code 4a__close__CNY_  \
0   1  2024-02-18    10.58500000                  ATOM     75.71983360   
1   2  2024-01-31    11.42200000                  ATOM     65.51930880   
2   3  2023-12-31    12.60000000                  ATOM     76.25216000   
3   4  2023-11-30    10.57900000                  ATOM     66.53360640   
4   5  2023-10-31     8.24900000                  ATOM     57.03805440   

  Digital_Currency_Name 1b__open__USD_           5__volume From_Symbol  \
0                Cosmos     9.10800000   42129750.45000000        None   
1                Cosmos    10.60000000   94200177.07999997        None   
2                Cosmos     9.24900000  152733350.39000002        None   
3                Cosmos     7.92700000  118290054.49000001        None   
4                Cosmos     7.23700000   56848395.96999999        None   

  To_Symbol 6__market_cap__USD_ 3a__low__CNY_ 3b__low__USD_ 2a__high__CNY_  \


In [8]:
df.dtypes

id                        int64
date                     object
2b__high__USD_           object
Digital_Currency_Code    object
4a__close__CNY_          object
Digital_Currency_Name    object
1b__open__USD_           object
5__volume                object
From_Symbol              object
To_Symbol                object
6__market_cap__USD_      object
3a__low__CNY_            object
3b__low__USD_            object
2a__high__CNY_           object
4b__close__USD_          object
1a__open__CNY_           object
dtype: object

In [9]:
import pandas as pd
import re
import warnings
import numpy as np

warnings.filterwarnings("ignore", message="Could not infer format")

def infer_data_types(df):
    # Define the percentage of rows to sample
    sample_percentage = 0.8
    
    # Sample the data
    sample = df.sample(frac=sample_percentage, random_state=42)
    
    # Dictionary to store inferred data types for each column
    inferred_data_types = {}
    
    # Loop through each column and infer the data type
    for col in sample.columns:
        inferred_data_type = infer_data_type(sample[col])
        inferred_data_types[col] = inferred_data_type
    
    return inferred_data_types

import pandas as pd
import numpy as np

def infer_data_type(series):
    if series.dtype == 'object':
        # Convert non-string values to strings
        series = series.astype(str)
        
        # Replace 'None' with NaN
        series = series.replace('None', np.nan).infer_objects(copy=False)
        
        # Remove empty strings and NaNs from the series
        non_empty_series = series.dropna()
        non_empty_series = non_empty_series[non_empty_series != '']
        # Convert all non-empty series elements to strings
        non_empty_series = non_empty_series.astype(str)
        
        # Check if the series contains mostly integers
        if non_empty_series.str.replace('-', '').str.isdigit().mean() > 0.8:
            return 'datetime.date'
        
        # Check if the series contains mostly floats
        elif non_empty_series.str.replace('.', '').str.replace('-', '').str.replace(',', '').str.replace('%', '').str.isdigit().mean() > 0.8:
            return 'float64'
        
        # Check if the series contains mostly dates
        elif non_empty_series.str.match(r'^\d{4}-\d{2}-\d{2}$').mean() > 0.8:
            return 'datetime.date'
        
        # Check if the series contains mostly currency values ('$XXX.XX')
        elif non_empty_series.str.startswith('$').mean() > 0.8:
            return 'float64'
        
        # Check if the series contains mostly years (4-digit numbers)
        elif non_empty_series.str.match(r'^\d{4}$').mean() > 0.8:
            return 'int64'
        
        # Check if the series contains mostly percentage values ('X.XX%')
        elif non_empty_series.str.match(r'^-?\d+\.\d+%$').mean() > 0.8:
            return 'float64'
        
        # If none of the above conditions are met, assume it's an object
        else:
            return 'object'
    elif series.dtype == 'datetime64[ns]':
        return 'datetime.date'
    else:
        return str(series.dtype)

def convert_columns(df, inferred_types):
    for col, dtype in inferred_types.items():
        if dtype == 'datetime.date':
            df[col] = df[col].astype(str)
            df[col] = df[col].replace('None', np.nan)
            df[col] = df[col].str.replace('-', '')
            df[col] = pd.to_datetime(df[col]).dt.date
        elif dtype == 'int64':
            # Apply cleaning operations similar to infer_data_type function
            df[col] = df[col].astype(str)
            df[col] = df[col].replace('None', np.nan)
            df[col] = df[col].str.replace('-', '')
            df[col] = pd.to_numeric(df[col]).fillna(0).astype(int)   
        elif dtype == 'float64':
            df[col] = df[col].astype(str)
            df[col] = df[col].replace('None', np.nan)
            df[col] = df[col].str.replace('-', '')
            df[col] = df[col].str.replace('.', '').str.replace('-', '').str.replace(',', '').str.replace('%', '').str.replace('$','')
            df[col] = pd.to_numeric(df[col]).fillna(0).astype(float)
        
        elif dtype == 'object':
            df[col] = df[col].astype(str)
        # Add more conditions for other data types if needed
    return df






# Example usage
# Replace df with your actual DataFrame
inferred_types = infer_data_types(df)
print("Inferred data types:")
print(inferred_types)



Inferred data types:
{'id': 'int64', 'date': 'datetime.date', '2b__high__USD_': 'float64', 'Digital_Currency_Code': 'object', '4a__close__CNY_': 'float64', 'Digital_Currency_Name': 'object', '1b__open__USD_': 'float64', '5__volume': 'float64', 'From_Symbol': 'object', 'To_Symbol': 'object', '6__market_cap__USD_': 'float64', '3a__low__CNY_': 'float64', '3b__low__USD_': 'float64', '2a__high__CNY_': 'float64', '4b__close__USD_': 'float64', '1a__open__CNY_': 'float64'}


C:\Users\yskon\AppData\Local\Temp\ipykernel_16528\394953751.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.replace('None', np.nan).infer_objects(copy=False)
C:\Users\yskon\AppData\Local\Temp\ipykernel_16528\394953751.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.replace('None', np.nan).infer_objects(copy=False)


In [10]:
# Assuming 'inferred_types' is a dictionary containing column names and their inferred types
df = convert_columns(df, inferred_types)

# Now df contains columns converted to the inferred data types
print(df.dtypes)

# Corresponding type of row in columns
for col, dtype in inferred_types.items():
    print(f"Column '{col}' corresponds to type '{dtype}'")


id                         int32
date                      object
2b__high__USD_           float64
Digital_Currency_Code     object
4a__close__CNY_          float64
Digital_Currency_Name     object
1b__open__USD_           float64
5__volume                float64
From_Symbol               object
To_Symbol                 object
6__market_cap__USD_      float64
3a__low__CNY_            float64
3b__low__USD_            float64
2a__high__CNY_           float64
4b__close__USD_          float64
1a__open__CNY_           float64
dtype: object
Column 'id' corresponds to type 'int64'
Column 'date' corresponds to type 'datetime.date'
Column '2b__high__USD_' corresponds to type 'float64'
Column 'Digital_Currency_Code' corresponds to type 'object'
Column '4a__close__CNY_' corresponds to type 'float64'
Column 'Digital_Currency_Name' corresponds to type 'object'
Column '1b__open__USD_' corresponds to type 'float64'
Column '5__volume' corresponds to type 'float64'
Column 'From_Symbol' corresponds to 

In [11]:
df.head()

,id,date,2b__high__USD_,Digital_Currency_Code,4a__close__CNY_,Digital_Currency_Name,1b__open__USD_,5__volume,From_Symbol,To_Symbol,6__market_cap__USD_,3a__low__CNY_,3b__low__USD_,2a__high__CNY_,4b__close__USD_,1a__open__CNY_
0,1,2024-02-18,1.058500e+09,ATOM,7.571983e+09,Cosmos,9.108000e+08,4.212975e+15,None,None,4.212975e+15,6.361300e+09,884300000.0,7.614426e+09,1.052600e+09,6.551931e+09
1,2,2024-01-31,1.142200e+09,ATOM,6.551931e+09,Cosmos,1.060000e+09,9.420018e+15,None,None,9.420018e+15,6.258432e+09,870000000.0,8.216530e+09,9.108000e+08,7.625216e+09
2,3,2023-12-31,1.260000e+09,ATOM,7.625216e+09,Cosmos,9.249000e+08,1.527334e+16,None,None,1.527334e+16,6.603005e+09,917900000.0,9.063936e+09,1.060000e+09,6.653361e+09
3,4,2023-11-30,1.057900e+09,ATOM,6.653361e+09,Cosmos,7.927000e+08,1.182901e+16,None,None,1.182901e+16,5.405271e+09,751400000.0,7.610109e+09,9.249000e+08,5.702367e+09
4,5,2023-10-31,8.249000e+08,ATOM,5.703805e+09,Cosmos,7.237000e+08,5.684840e+15,None,None,5.684840e+15,4.429819e+09,615800000.0,5.934001e+09,7.929000e+08,5.206008e+09


In [13]:
# Using dtype attribute for a single column
data_type = df["4a__close__CNY_"].dtype
print("Data type of column '4a__close__CNY_':", data_type)




Data type of column '4a__close__CNY_': float64
